<a href="https://colab.research.google.com/github/shreya0202/PyCuda/blob/main/Matrix_Multiplication_Pycuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda # install cuda

     |████████████████████████████████| 1.6MB 8.9MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620899 sha256=f184cce504bd00590402e7bc19c51f435da774991078344dc028504f2b1baef9
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=3ea1553fdd8f9066b9e041e19e91c4678a425d5fe825b6b7797bcadd1db33fa8
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


Python: Assignment 4 for case A ∈ R500×500, B ∈ R500×400, N > 100

In [ ]:
import numpy as np
import time
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit

gpu_code = """
__global__ void MatrixMulKernel(int N, double *A, double *B, double *C)
{
    int threadId= (blockDim.x * blockDim.y) * threadIdx.z + (threadIdx.y * blockDim.x) + (threadIdx.x);
    int blockId= (gridDim.x * gridDim.y) * blockIdx.z + (blockIdx.y * gridDim.x) + (blockIdx.x);
    int DimBlock= blockDim.x * blockDim.y * blockDim.z;
    int DimGrid= gridDim.x * gridDim.y * gridDim.z;

    for (int i = blockId; i < N; i += DimGrid)
    {
      for(int j = threadId; j < N; j += DimBlock)
      {
        for(int k = 0; k < N; k++)
        { 
          C [i * N +j] += ( A [ i * N + k] * B [ k * N + j] );
        }
      }
    }

}
"""

def compute(x, y, l, m, N) :

      start_time = time.time()

      # Randomly assign values to matrices
      A_H = np.random.randn(x, y).astype(np.float32)
      B_H = np.random.randn(l, m).astype(np.float32)

      # Host Matrix Multiplication
      C_H = np.dot(A_H, B_H)
      print("CPU Time for N = %d is: %.7s secs" % (N, (time.time() - start_time)) )

      # Transfer host to kernel
      d_A = gpuarray.to_gpu(A_H) 
      d_B = gpuarray.to_gpu(B_H)

      d_C = gpuarray.empty((x, m), np.float32)

      # Assigning N
      gpuCode = gpu_code % {
          'N': N 
          }

      out = compiler.SourceModule(gpuCode)
      matrixmulkernel = out.get_function("MatrixMulKernel")
      matrixmulkernel(d_A, d_B,d_C, block = (16, 16, 1), grid = (16,8,1))

      print("GPU Time for N = %d is: %.7s secs" % (N, (time.time() - start_time)) )
      print()


def main():
  print("Python Results for Assignment 4:")
  print()
  switcher = { 
      1: compute(500, 500, 500, 400, 150), 
      2: compute(500, 500, 500, 400, 300), 
      3: compute(500, 500, 500, 400, 500), 
  } 

main()



Python Results for Assignment 4:

CPU Time for N = 150 is: 0.02667 secs
GPU Time for N = 150 is: 0.02942 secs

CPU Time for N = 300 is: 0.02685 secs
GPU Time for N = 300 is: 0.02984 secs

CPU Time for N = 500 is: 0.02596 secs
GPU Time for N = 500 is: 0.02826 secs



Python: Assignment 3 Part II

In [ ]:
import numpy as np
import time
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit

gpu_code = """
__global__ void MatrixMulKernel(int N, double *A, double *B, double *C)
{
    int threadId= (blockDim.x * blockDim.y) * threadIdx.z + (threadIdx.y * blockDim.x) + (threadIdx.x);
    int blockId= (gridDim.x * gridDim.y) * blockIdx.z + (blockIdx.y * gridDim.x) + (blockIdx.x);
    int DimBlock= blockDim.x * blockDim.y * blockDim.z;
    int DimGrid= gridDim.x * gridDim.y * gridDim.z;

    for (int i = blockId; i < N; i += DimGrid)
    {
      for(int j = threadId; j < N; j += DimBlock)
      {
        for(int k = 0; k < N; k++)
        { 
          C [i * N +j] += ( A [ i * N + k] * B [ k * N + j] );
        }
      }
    }

}
"""

def compute(x, y, l, m, N, num) :

      start_time = time.time()

      # Randomly assign values to matrices
      A_H = np.random.randn(x, y).astype(np.float32)
      B_H = np.random.randn(l, m).astype(np.float32)

      # Host Matrix Multiplication
      C_H = np.dot(A_H, B_H)
      print("CPU Time for case %d is: %.7s secs" % (num, (time.time() - start_time)) )

      # Transfer host to kernel
      d_A = gpuarray.to_gpu(A_H) 
      d_B = gpuarray.to_gpu(B_H)

      d_C = gpuarray.empty((x, m), np.float32)

      # Assigning N
      gpuCode = gpu_code % {
          'N': N 
          }

      out = compiler.SourceModule(gpuCode)
      matrixmulkernel = out.get_function("MatrixMulKernel")
      matrixmulkernel(d_A, d_B,d_C, block = (16, 16, 1), grid = (16,8,1))

      print("GPU Time for case %d is: %.7s secs" % (num, (time.time() - start_time)) )
      print()


def main():
  print("Python results for Assignmnet 3-Part II:")
  print()
  switcher = { 
      1: compute(500, 500, 500, 400, 100, 1), 
      2: compute(50, 20, 20, 50, 5000, 2), 
      3: compute(6, 4000, 4000, 9, 1000, 3), 
  } 

main()


Python results for Assignmnet 3-Part II:

CPU Time for case 1 is: 0.02635 secs
GPU Time for case 1 is: 0.02997 secs

CPU Time for case 2 is: 0.00047 secs
GPU Time for case 2 is: 0.00219 secs

CPU Time for case 3 is: 0.00339 secs
GPU Time for case 3 is: 0.00503 secs

